In [1]:
import torch
import optuna
import optuna.visualization as vis
from torch.utils.tensorboard import SummaryWriter

from utils.metrices import roc_auc
from utils.read_data import IMDB_mlh
from MARA.MARA import MARA

from config import config

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Working on device: ", device)

imdb = IMDB_mlh().to(device)
imdb.info()

train_mask, val_mask, test_mask = imdb.get_training_mask(train_mask_size=0.25, val_mask_size=0.25)

writer = SummaryWriter(log_dir="./tensorboard_logs")

/home/lewy700/Documents/MARA/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-08 02:38:46.105060: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 02:38:46.190665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733621926.295792   10193 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733621926.327921   10193 cuda_blas.cc:1

MARA(
  (conv1): GCNConv(1000, 512)
  (conv2): GCNConv(512, 256)
  (conv3): GCNConv(256, 52)
  (classifier): Linear(in_features=52, out_features=3, bias=True)
  (ReLU): ReLU6()
  (dropout): Dropout(p=0.001, inplace=False)
  (dropedge): DropEdge()
  (neuralsparse_1): NeuralSparse(
    (mlp): Sequential(
      (0): Linear(in_features=4000, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (neuralsparse_2): NeuralSparse(
    (mlp): Sequential(
      (0): Linear(in_features=4000, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (neuralsparse_3): NeuralSparse(
    (mlp): Sequential(
      (0): Linear(in_features=4000, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=1, bias=True)
    )
  )
)
Working on device:  cuda
IMDB movie type dataset:
 Number of nodes: 5614
 Number of edges: 14715
 Number of edges: 

In [2]:
def train(data, model, criterion, optimizer, epoch, trial_number):
    model.train()
    optimizer.zero_grad()

    intra_layer_edges = [data.layer_1, data.layer_2]
    cross_layer_edges = [data.cross_edges]

    out, _, _ = model(data.node_features, intra_layer_edges, cross_layer_edges)

    train_loss = criterion(out[train_mask], data.classes[train_mask])
    train_loss.backward()
    optimizer.step()

    train_score = roc_auc(out[train_mask], data.classes[train_mask])
    val_score = roc_auc(out[val_mask], data.classes[val_mask])

    writer.add_scalar(f"Trial {trial_number}/Loss/train", train_loss.item(), epoch)
    writer.add_scalar(f"Trial {trial_number}/ROC-AUC/train", train_score, epoch)
    writer.add_scalar(f"Trial {trial_number}/ROC-AUC/val", val_score, epoch)

    return train_loss.item(), train_score, val_score

In [3]:
def objective(trial):
    params = {
        "simplification_type": "l-b-l",
        "simplification_stages": "each",
        "DE_p": trial.suggest_float("DE_p", 0, 1),
        "lr": trial.suggest_float("lr", 1e-5, 1e-2, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True),
        "dropout": trial.suggest_float("dropout", 0, 0.7)
    }

    mara = MARA(
        simplification_type=params["simplification_type"],
        simplification_stages=params["simplification_stages"],
        DE_p=params["DE_p"],
        dropout=params["dropout"]
    ).to(device)
    crit = torch.nn.CrossEntropyLoss()
    optim = torch.optim.Adam(mara.parameters(), lr=params["lr"], weight_decay=params["weight_decay"])

    early_stopping = {
        "best_val_score": 0,
        "patience": config["patience"],
        "counter": 0,
        "best_weights": None
    }

    for epoch in range(config["epoch_num"]):
        train_loss, train_score, val_score = train(imdb, mara, crit, optim, epoch, trial.number)

        if val_score > early_stopping["best_val_score"]:
            early_stopping = {
                "best_val_score": val_score,
                "counter": 0,
                "best_weights": mara.state_dict(),
                "patience": early_stopping["patience"]
            }
        else:
            early_stopping["counter"] += 1

        if early_stopping["counter"] >= early_stopping["patience"]:
            break

    mara.load_state_dict(early_stopping["best_weights"])

    trial_number = trial.number
    writer.add_scalar(f"Trial {trial_number}/Best ROC-AUC/val", early_stopping["best_val_score"], trial_number)
    for param_name, param_value in params.items():
        writer.add_text(f"Trial {trial_number}/Params", f"{param_name}: {param_value}", trial_number)

    return early_stopping["best_val_score"]

In [4]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=3)

[I 2024-12-08 02:38:58,138] A new study created in memory with name: no-name-3cb4a5fe-222d-4da1-9acc-fc6a5591771b
[I 2024-12-08 02:39:07,711] Trial 0 finished with value: 0.585864924965683 and parameters: {'DE_p': 0.5164798480558577, 'lr': 4.398282201978791e-05, 'weight_decay': 0.0006800846125472352, 'dropout': 0.4929031825261195}. Best is trial 0 with value: 0.585864924965683.
[I 2024-12-08 02:39:14,677] Trial 1 finished with value: 0.6654298744918148 and parameters: {'DE_p': 0.5033306679993136, 'lr': 5.161955883231108e-05, 'weight_decay': 3.0225400937387605e-06, 'dropout': 0.061205818025099376}. Best is trial 1 with value: 0.6654298744918148.
[I 2024-12-08 02:39:24,103] Trial 2 finished with value: 0.8082534516707528 and parameters: {'DE_p': 0.31457299462561483, 'lr': 0.0008392710501866806, 'weight_decay': 1.0141522251830408e-06, 'dropout': 0.03681301686999636}. Best is trial 2 with value: 0.8082534516707528.


In [5]:
best_trial = study.best_trial

print("Best trial:")
print(f"  Value: {best_trial.value}")
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Value: 0.8082534516707528
  Params: 
    DE_p: 0.31457299462561483
    lr: 0.0008392710501866806
    weight_decay: 1.0141522251830408e-06
    dropout: 0.03681301686999636


In [6]:
vis.plot_slice(study)

In [7]:
vis.plot_optimization_history(study)

In [8]:
vis.plot_param_importances(study)